In [1]:
import numpy as np
import os 
import re
import random
import pandas as pd
import subprocess
import shutil
import time
import sys

from pymatgen.core import Element
from pymatgen.io.vasp import Poscar
from pymatgen.io.cif import CifWriter
from pymatgen.core import Composition
from pymatgen.analysis.structure_matcher import StructureMatcher

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from matplotlib.patches import Rectangle
from scipy.cluster.hierarchy import dendrogram, linkage

In [2]:
#print per ogni coppia il ground state e l'entalpia del template che prendo

#Needs df of Individuals, fitness treshold, file gatheredPoscars and return best non duplicated structures (with symm>75)
def best_structures(individuals_df, fitness_upto, all_poscars):
    fitness_gs = individuals_df['fitness'].iloc[0]
    uniques = []
    SGs=[]
    fitness = []
    structure_gs = Poscar.from_str(find_poscar(all_poscars, individuals_df['ID'].iloc[0]))
    
    if (individuals_df['spacegroup'].iloc[0] > 75):
        uniques.append(structure_gs)
        SGs.append(individuals_df['spacegroup'].iloc[0])
        fitness.append(individuals_df['fitness'].iloc[0])

    for i, line_individuals_df in individuals_df.iterrows():
        if line_individuals_df['fitness'] - fitness_gs >= fitness_upto:
            break
        if line_individuals_df['spacegroup'] < 75:
            continue
        new_structure = Poscar.from_str(find_poscar(all_poscars, line_individuals_df['ID']))

        check_dupilcate = False
        
        if StructureMatcher(ltol = 1.0, stol = 1.0, angle_tol = 10, scale=True).fit(structure_gs.structure, new_structure.structure):
            check_dupilcate = True 
        
        for structure in uniques:
            if StructureMatcher(ltol = 1.0, stol = 1.0, angle_tol = 10, scale=True).fit(structure.structure, new_structure.structure):
                check_dupilcate = True
        if not check_dupilcate:
            uniques.append(new_structure)
            SGs.append(line_individuals_df['spacegroup'])
            fitness.append(line_individuals_df['fitness'])
    return uniques, SGs, fitness

def read_individuals(individuals):
    column_names = {0 : "Generation", 1 : "ID", 2 : "GenMode", 
    4 : 'A', 5 : 'B', 7 : "enthalpy", 10 : "fitness", 15 : "spacegroup"}

    individuals = pd.read_csv(individuals, sep="\s+", header=None, skiprows=2,usecols=column_names)
    individuals.rename(columns=column_names, inplace=True)
    individuals.sort_values("fitness", inplace=True)
    return individuals

#Needs in input the name of the file with all the poscars, the id of the structure to be extracted and the name of the output poscar
def find_poscar(all_poscars, id):
    end=-1
    with open(all_poscars,'r') as file:
        testo_input = file.readlines()              
    for i, line in enumerate(testo_input):
        if line.startswith('EA'+str(id)):   
            init = i
            simm=int(line[line.find(':')+1:])
        if line.startswith('EA'+str(id+1)):
            end = i-1
            break
    if end == -1:                            
        end = len(testo_input)-1
        
    poscar_str=''
    for i in range(end-init+1):
        poscar_str+=testo_input[init+i]
    return poscar_str

In [3]:
test_elements = ['Na', 'N', 'B', 'Be', 'C' , 'Mg', 'O', 'Li']
for i in range(len(test_elements)):
    for j in range(i+1,len(test_elements)):
        cp = [test_elements[i],test_elements[j]]
        cp.sort()
        A = cp[0]
        B = cp[1]
        df = read_individuals(f'./all_Individuals/{A+B}_Individuals')
        _ , sg_temp, fitness_temp = best_structures(df, 0.1,f'./all_poscars/{A+B}_gatheredPOSCARS')
        print(f'COUPLE: {A+B} GS_ENT: {df["fitness"].iloc[0]}, SG: {df["spacegroup"].iloc[0]}, SG_temp: {sg_temp}, GS_temp: {fitness_temp}')

COUPLE: NNa GS_ENT: -1519.988, SG: 65, SG_temp: [], GS_temp: []
COUPLE: BNa GS_ENT: -1316.551, SG: 74, SG_temp: [], GS_temp: []
COUPLE: BeNa GS_ENT: -1615.006, SG: 59, SG_temp: [], GS_temp: []
COUPLE: CNa GS_ENT: -1400.666, SG: 2, SG_temp: [], GS_temp: []
COUPLE: MgNa GS_ENT: -2849.078, SG: 59, SG_temp: [123], GS_temp: [-2849.033]
COUPLE: NaO GS_ENT: -1687.852, SG: 65, SG_temp: [], GS_temp: []
COUPLE: LiNa GS_ENT: -1430.422, SG: 187, SG_temp: [187, 194, 123], GS_temp: [-1430.422, -1430.369, -1430.363]
COUPLE: BN GS_ENT: -364.527, SG: 216, SG_temp: [216, 186], GS_temp: [-364.527, -364.486]
COUPLE: BeN GS_ENT: -662.59, SG: 12, SG_temp: [], GS_temp: []
COUPLE: CN GS_ENT: -444.956, SG: 166, SG_temp: [166, 164], GS_temp: [-444.956, -444.924]
COUPLE: MgN GS_ENT: -1898.471, SG: 187, SG_temp: [187], GS_temp: [-1898.471]
COUPLE: NO GS_ENT: -728.617, SG: 160, SG_temp: [160], GS_temp: [-728.617]
COUPLE: LiN GS_ENT: -478.261, SG: 47, SG_temp: [139], GS_temp: [-478.233]
COUPLE: BBe GS_ENT: -459.276

In [5]:
test_elements = ['Na', 'N', 'B', 'Be', 'C' , 'Mg', 'O', 'Li']
df_gs = pd.read_csv('./GroundStates_3.txt')

for i in range(len(test_elements)):
    for j in range(i+1,len(test_elements)):
        cp = [test_elements[i],test_elements[j]]
        cp.sort()
        A = cp[0]
        B = cp[1]
        df = read_individuals(f'./all_Individuals/{A+B}_Individuals')    
        print(f'{A+B} GS: {df["fitness"].iloc[0]/2}, GS_temp: {df_gs[df_gs["COUPLES"] == f"{A}{B}"]["ENT"].values[0]}')
        

NNa GS: -759.994, GS_temp: -759.6513620769399
BNa GS: -658.2755, GS_temp: -657.9911059083599
BeNa GS: -807.503, GS_temp: -807.16357166736
CNa GS: -700.333, GS_temp: -700.04496709822
MgNa GS: -1424.539, GS_temp: -1423.86662886176
NaO GS: -843.926, GS_temp: -843.5751569601599
LiNa GS: -715.211, GS_temp: -714.87153031028
BN GS: -182.2635, GS_temp: -180.52723081924
BeN GS: -331.295, GS_temp: 0.0
CN GS: -222.478, GS_temp: -220.65430443231327
MgN GS: -949.2355, GS_temp: -946.25270189054
NO GS: -364.3085, GS_temp: -361.88590448290665
LiN GS: -239.1305, GS_temp: -237.07238967798003
BBe GS: -229.638, GS_temp: -227.62597050428
BC GS: -121.6385, GS_temp: -119.79770681442
BMg GS: -847.709, GS_temp: -844.5314877292
BO GS: -265.5015, GS_temp: -263.4280666556
BLi GS: -137.856, GS_temp: -135.92344253274666
BeC GS: -271.334, GS_temp: 0.0
BeMg GS: -997.1555, GS_temp: -993.65944733112
BeO GS: -416.382, GS_temp: -414.37249306916
BeLi GS: -287.3045, GS_temp: -284.9594751288
CMg GS: -889.313, GS_temp: -885.

In [6]:
test_elements = ['Na', 'N', 'B', 'Be', 'C' , 'Mg', 'O', 'Li']
df_gs = pd.read_csv('./GroundStates_3.txt')
df_rel = pd.read_csv('./AB_relaxation/RELAX_DATA_Merged', index_col=0, header=0)


for i in range(len(test_elements)):
    for j in range(i+1,len(test_elements)):
        cp = [test_elements[i],test_elements[j]]
        cp.sort()
        A = cp[0]
        B = cp[1]
        df = read_individuals(f'./all_Individuals/{A+B}_Individuals')   
        _ , sg_temp, fitness_temp = best_structures(df, 0.1,f'./all_poscars/{A+B}_gatheredPOSCARS')
        if (len(sg_temp) == 0):
            continue
        sg = int(sg_temp[0])
        fit = fitness_temp[0]
        print(A+B,sg,df_rel.loc[f'{A}{B}',f'{A}{B}_{sg}']-df_gs[df_gs["COUPLES"] == f"{A}{B}"]["ENT"].values[0])
        

MgNa 123 0.08477834992004318
LiNa 187 -0.00035929499995290826
BN 216 1.281120006524361e-06
CN 166 -0.00010728654672220728
MgN 187 222.27951658119343
NO 160 0.000606324719967688
LiN 139 -0.09951566827996317
BBe 164 -0.000677367720015809
BC 139 1.5860999980077395e-06
BLi 166 0.05898964515333205
BeO 186 0.0008471677999750682
CLi 164 -3.073362000804991e-05
MgO 225 5.918175997976505e-05
LiMg 123 -0.00939316299991333
LiO 187 -41.333537911000064


In [8]:
test_elements = ['Na', 'N', 'B', 'Be', 'C' , 'Mg', 'O', 'Li']
df_gs = pd.read_csv('./GroundStates_3.txt')
df_gs_1 = pd.read_csv('./GroundStates_2.txt')


for i in range(len(test_elements)):
    for j in range(i+1,len(test_elements)):
        cp = [test_elements[i],test_elements[j]]
        cp.sort()
        A = cp[0]
        B = cp[1]
        df = read_individuals(f'./all_Individuals/{A+B}_Individuals')   
        _ , sg_temp, fitness_temp = best_structures(df, 0.1,f'./all_poscars/{A+B}_gatheredPOSCARS')
        if (len(sg_temp) == 0):
            continue
        sg = int(sg_temp[0])
        fit = fitness_temp[0]
        print(A+B,sg,df_gs_1[df_gs_1["COUPLES"] == f"{A}{B}"]["ENT"].values[0]-df_gs[df_gs["COUPLES"] == f"{A}{B}"]["ENT"].values[0])
        

MgNa 123 -0.012371756799893774
LiNa 187 -0.00014590080002108152
BN 216 2.4480016236338997e-08
CN 166 -2.762885335982901e-05
MgN 187 222.28003785264661
NO 160 1.2992533697797626e-06
LiN 139 2.0325200296156254e-06
BBe 164 -0.0003907337800228561
BC 139 -2.0274200096537243e-06
BLi 166 -7.532133281529241e-07
BeO 186 -3.5190000176044123e-06
CLi 164 -3.656427995224476e-05
MgO 225 4.019684001832502e-05
LiMg 123 -0.009395168999958514
LiO 187 -41.333919767946725
